<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Mistral_Context_Engineering_Agent_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab_env -q
!pip install mistralai -q

In [2]:
import json
import uuid # For generating unique IDs, e.g., for user sessions or documents
import os
import asyncio # Required for async operations

# Ensure colab_env is available if running in Colab, otherwise it might not be needed
try:
    import colab_env
except ImportError:
    print("Warning: 'colab_env' not found. If running in Colab, please install it.")
    pass

from mistralai import Mistral

# --- Mistral API Configuration ---
# It's crucial to set your MISTRAL_API_KEY as an environment variable.
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    # This print statement will be visible in the console if the key is missing.
    # In a real application, you might raise an exception or handle it gracefully.
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    client = None
else:
    client = Mistral(api_key=api_key)

MISTRAL_MODEL = "open-mixtral-8x22b"

# --- Mistral API Interaction Function ---
async def call_mistral_api(prompt_text):
    """
    Makes a call to the Mistral API using the configured client.

    Args:
        prompt_text (str): The text prompt to send to the LLM.

    Returns:
        dict: A dictionary containing the LLM's response text, or an error message.
    """
    if client is None:
        print("Mistral client not initialized due to missing API key. Cannot make API call.")
        return {"text": "Error: Mistral API key not set."}

    print(f"\n--- Calling Mistral API ({MISTRAL_MODEL}) ---")
    print(f"Prompt: {prompt_text[:200]}...") # Print first 200 chars of prompt

    messages = [{"role": "user", "content": prompt_text}]

    try:
        chat_response = client.chat.complete(
            model=MISTRAL_MODEL,
            messages=messages, # Using a dictionary for the message
        )
        response_content = chat_response.choices[0].message.content
        print(f"Mistral Response: {response_content[:200]}...") # Print first 200 chars of response
        return {"text": response_content}
    except Exception as e:
        print(f"Error calling Mistral API: {e}")
        return {"text": f"Error: Failed to get response from Mistral API: {e}"}

# --- Agent Class Definition ---

class MistralContextEngineerAgent:
    """
    Represents an AI agent structured around the "Context Engineering" framework.
    """

    def __init__(self, agent_persona="A clear-thinking, structured assistant"):
        """
        Initializes the Mistral Context Engineer Agent.

        Args:
            agent_persona (str): A description of the agent's role or persona.
        """
        self.agent_id = str(uuid.uuid4())
        self.agent_persona = agent_persona
        self.objective = None
        self.context_package = {
            "audience": None,
            "voice_tone": None,
            "length_target": None,
            "key_facts_sources": [],
            "known_constraints_boundaries": []
        }
        self.draft = ""
        self.workflow_step = 0
        self.user_feedback = ""
        self.is_agreed = False
        self.output_format = "plain text"
        self.chat_history = [] # To keep track of conversation for LLM context

        print(f"Mistral Context Engineer Agent initialized with ID: {self.agent_id}") # Changed print statement
        print(f"Agent Persona: {self.agent_persona}")

    def set_objective(self, objective_description):
        """
        Sets the primary objective for the agent.

        Args:
            objective_description (str): A clear description of the desired outcome.
        """
        self.objective = objective_description
        self._add_to_history(f"Objective set: {objective_description}")
        print(f"Objective set: {self.objective}")

    def add_context(self, context_type, value):
        """
        Adds information to the agent's context package.

        Args:
            context_type (str): The type of context (e.g., 'audience', 'voice_tone',
                                'length_target', 'key_fact', 'constraint').
            value (str or list): The context information.
        """
        if context_type == "key_fact":
            self.context_package["key_facts_sources"].append(value)
            print(f"Added key fact: {value}")
        elif context_type == "constraint":
            self.context_package["known_constraints_boundaries"].append(value)
            print(f"Added constraint: {value}")
        elif context_type in self.context_package:
            self.context_package[context_type] = value
            print(f"Set {context_type}: {value}")
        else:
            print(f"Warning: Unknown context type '{context_type}'.")
        self._add_to_history(f"Context added: {context_type} - {value}")

    async def _perform_gap_check(self):
        """
        Performs the initial "Gap Check" workflow step.
        Asks the LLM to identify missing information.
        """
        prompt = f"""
        As a {self.agent_persona}, your current objective is: "{self.objective}".
        Here is the current context package:
        Audience: {self.context_package['audience']}
        Voice/Tone: {self.context_package['voice_tone']}
        Length Target: {self.context_package['length_target']}
        Key Facts/Sources: {', '.join(self.context_package['key_facts_sources'])}
        Constraints/Boundaries: {', '.join(self.context_package['known_constraints_boundaries'])}

        Based on this, perform a "Gap check". List any information still missing that is crucial
        for achieving the objective, and ask concise questions to fill these gaps.
        """
        response = await call_mistral_api(prompt)
        print(f"\n--- Gap Check Result ---")
        print(response['text'])
        self._add_to_history(f"Gap Check: {response['text']}")
        return response['text']

    async def _outline_structure(self):
        """
        Asks the LLM to outline a logical structure for the output.
        """
        prompt = f"""
        As a {self.agent_persona}, your current objective is: "{self.objective}".
        Considering the following context:
        {json.dumps(self.context_package, indent=2)}

        Please outline a logical structure or bullet agenda for the final output.
        """
        response = await call_mistral_api(prompt)
        print(f"\n--- Outline Structure ---")
        print(response['text'])
        self._add_to_history(f"Outline: {response['text']}")
        return response['text']

    async def _draft_version(self):
        """
        Asks the LLM to draft a version of the output.
        """
        prompt = f"""
        As a {self.agent_persona}, your current objective is: "{self.objective}".
        Considering the following context:
        {json.dumps(self.context_package, indent=2)}
        And the current workflow step: {self.workflow_step}

        Please draft a version of the output based on the current plan and context.
        Current draft (if any): {self.draft[:100]}...
        """
        response = await call_mistral_api(prompt)
        self.draft = response['text']
        print(f"\n--- Draft Version (v{self.workflow_step}) ---")
        print(self.draft[:500] + "...") # Print first 500 chars of draft
        self._add_to_history(f"Draft v{self.workflow_step}: {self.draft[:100]}...")
        return self.draft

    async def _handle_large_input(self, input_text):
        """
        Simulates handling large input sources as per context rules.
        """
        if len(input_text.split()) > 200: # Simple word count check
            prompt = f"The provided source text exceeds 200 words. Please give me a one-sentence summary of this text: '{input_text[:500]}...' and ask if I want to process the full text."
            response = await call_mistral_api(prompt)
            print(f"\n--- Large Input Handling ---")
            print(response['text'])
            self._add_to_history(f"Large Input: {response['text']}")
            return response['text']
        return None

    async def run_workflow(self):
        """
        Executes the iterative workflow for the agent.
        This simulates the interaction and feedback loop.
        """
        print("\n--- Starting Agent Workflow ---")
        self.workflow_step = 0
        self.is_agreed = False

        # Step 0: Gap Check
        print(f"\nWorkflow Step {self.workflow_step}: Performing Gap Check...")
        await self._perform_gap_check()
        # In a real scenario, the agent would wait for user input to fill gaps here.
        # For simulation, we'll assume gaps are filled or ignored for now.
        self.workflow_step += 1

        # Step 1: Outline Structure
        print(f"\nWorkflow Step {self.workflow_step}: Outlining Structure...")
        await self._outline_structure()
        self.workflow_step += 1

        # Step 2: First Version Draft
        print(f"\nWorkflow Step {self.workflow_step}: Drafting First Version...")
        await self._draft_version()
        self.workflow_step += 1

        # Simulate iterative feedback loop
        while not self.is_agreed and self.workflow_step < 5: # Limit iterations for simulation
            print(f"\nWorkflow Step {self.workflow_step}: Pausing for Feedback...")
            # In a real app, this would be where user provides feedback
            self.user_feedback = input("Please provide feedback on the draft (or type 'AGREE' to finalize): ").strip()
            self._add_to_history(f"User Feedback: {self.user_feedback}")

            if self.user_feedback.upper() == "AGREE":
                self.is_agreed = True
                print("User agreed. Finalizing draft.")
            else:
                print(f"Improving draft based on feedback: '{self.user_feedback}'")
                # Simulate improving the draft based on feedback
                prompt = f"""
                As a {self.agent_persona}, your current objective is: "{self.objective}".
                You are currently at workflow step {self.workflow_step}.
                Here is the current draft: {self.draft}
                The user has provided the following feedback: "{self.user_feedback}"

                Please improve the draft based on this feedback.
                """
                response = await call_mistral_api(prompt)
                self.draft = response['text']
                print(f"\n--- Improved Draft (v{self.workflow_step}) ---")
                print(self.draft[:500] + "...")
                self._add_to_history(f"Improved Draft v{self.workflow_step}: {self.draft[:100]}...")
                self.workflow_step += 1

        if not self.is_agreed:
            print("\nWorkflow completed without explicit agreement (simulated limit reached).")
        else:
            print("\nWorkflow completed successfully with user agreement.")

    async def generate_output(self):
        """
        Generates the final output based on the agreed-upon draft and specified format.
        """
        if not self.draft:
            print("Error: No draft available to generate output.")
            return None

        prompt = f"""
        As a {self.agent_persona}, your objective was: "{self.objective}".
        The final agreed-upon draft is: {self.draft}
        The desired output format is: "{self.output_format}"

        Please format the final output according to the specified format.
        If key facts were referenced, ensure they are cited by their list number from the context package.
        """
        response = await call_mistral_api(prompt)
        final_output = response['text']
        print(f"\n--- Final Output ({self.output_format}) ---")
        print(final_output)
        self._add_to_history(f"Final Output: {final_output[:100]}...")
        return final_output

    def _add_to_history(self, message):
        """Adds a message to the agent's internal chat history."""
        self.chat_history.append(message)

    def get_chat_history(self):
        """Returns the agent's internal chat history."""
        return self.chat_history

# --- Example Usage ---
async def main():
    agent = MistralContextEngineerAgent( # Changed agent instantiation
        agent_persona="A flight planning specialist AI, Mistral-powered edition" # Changed persona
    )

    # Set Objective
    agent.set_objective("Draft a detailed flight plan from CYUL to KLAX for a Boeing 787, adhering to ETOPS regulations and minimizing fuel consumption.")

    # Add Context
    agent.add_context("audience", "Airline Operations Team")
    agent.add_context("voice_tone", "Formal and Technical")
    agent.add_context("length_target", "2-3 pages")
    agent.add_context("key_fact", "CYUL (Montreal) to KLAX (Los Angeles)")
    agent.add_context("key_fact", "Aircraft: Boeing 787")
    agent.add_context("key_fact", "Adhere to ETOPS regulations")
    agent.add_context("constraint", "Minimize fuel consumption")
    agent.add_context("constraint", "Avoid restricted airspace R-2901")

    # Simulate a large input source (e.g., a regulatory document)
    large_doc_text = "This is a very long document about ETOPS regulations, detailing various requirements for twin-engine aircraft operations over extended ranges. It covers fuel reserves, diversion airports, maintenance procedures, and crew training. The document specifies that for a Boeing 787, ETOPS 330 is typically approved, meaning it can fly up to 330 minutes from a suitable diversion airport. This includes considerations for single-engine performance and depressurization scenarios. The document also provides guidelines on how to calculate adequate fuel for different phases of flight, including contingency fuel, alternate fuel, and final reserve fuel. Furthermore, it outlines the necessary communication and navigation equipment for ETOPS flights, emphasizing redundancy. The procedures for pre-flight planning and in-flight monitoring are also extensively detailed, ensuring safety and compliance. The document also touches upon the importance of real-time weather monitoring for all potential diversion airports along the planned route. Any changes to the flight plan due to weather or operational issues must be immediately communicated to air traffic control and the airline's operations center. This ensures that all parties are aware of the current status and can provide necessary support. The document concludes with a section on post-flight reporting requirements for ETOPS operations, including any deviations or incidents."
    await agent._handle_large_input(large_doc_text)
    # In a real scenario, the user would then confirm if they want to process the full text.
    # For this simulation, we'll assume they do.

    # Run the Workflow
    await agent.run_workflow()

    # Generate Final Output
    agent.output_format = "Markdown with H2 headings"
    await agent.generate_output()

    print("\n--- Agent Interaction History ---")
    for msg in agent.get_chat_history():
        print(msg)

# To run the async main function in a Colab environment, await it directly.
await main()

Mistral Context Engineer Agent initialized with ID: 35f3f528-a486-4a6f-8439-f064f36309a7
Agent Persona: A flight planning specialist AI, Mistral-powered edition
Objective set: Draft a detailed flight plan from CYUL to KLAX for a Boeing 787, adhering to ETOPS regulations and minimizing fuel consumption.
Set audience: Airline Operations Team
Set voice_tone: Formal and Technical
Set length_target: 2-3 pages
Added key fact: CYUL (Montreal) to KLAX (Los Angeles)
Added key fact: Aircraft: Boeing 787
Added key fact: Adhere to ETOPS regulations
Added constraint: Minimize fuel consumption
Added constraint: Avoid restricted airspace R-2901

--- Starting Agent Workflow ---

Workflow Step 0: Performing Gap Check...

--- Calling Mistral API (open-mixtral-8x22b) ---
Prompt: 
        As a A flight planning specialist AI, Mistral-powered edition, your current objective is: "Draft a detailed flight plan from CYUL to KLAX for a Boeing 787, adhering to ETOPS regulations and m...
Mistral Response: To comp